In [1]:
import pandas as pd
import numpy as np
import re

# scikit-learn
# -----------------------
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score

In [2]:
fake_df=pd.read_csv('../data/Fake.csv')
true_df=pd.read_csv('../data/True.csv')

In [3]:
fake_df.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
fake_df.info(), true_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
dtypes: object(4)
memory usage: 733.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21417 non-null  object
 1   text     21417 non-null  object
 2   subject  21417 non-null  object
 3   date     21417 non-null  object
dtypes: object(4)
memory usage: 669.4+ KB


(None, None)

In [5]:
fake_df.drop(['subject', 'date'], axis=1, inplace=True)
true_df.drop(['subject', 'date'], axis=1, inplace=True)

In [6]:
fake_df['class'] = 0
true_df['class'] = 1

In [7]:
news_df = pd.concat([fake_df, true_df], ignore_index=True, sort=False)

In [8]:
news_df['text'] = news_df['title'] + news_df['text']
news_df.drop(['title'], axis=1, inplace=True)

In [9]:
def wordopt(text):
    text = text.lower()                               # Convert to lowercase
    text = re.sub(r'https?://\S+|www\.\S+', '', text) # Remove URLs
    text = re.sub(r'\W', ' ', text)                   # Replace non-word characters with a space
    text = re.sub(r'\n', '', text)                    # Remove newline characters
    text = re.sub(r' +', ' ', text)                   # Replace multiple spaces with a single space
    text = re.sub(r'^ ', '', text)                    # Remove leading space
    text = re.sub(r' $', '', text)                    # Remove trailing space
    return text

news_df['text'] = news_df['text'].apply(wordopt)

In [10]:
X = news_df['text']
y = news_df['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=24)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
vect_X_train = vectorization.fit_transform(X_train)
vect_X_test = vectorization.transform(X_test)

In [12]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(vect_X_train,y_train)

LogisticRegression()

In [13]:
lr_y_pred = lr.predict(vect_X_test)


In [14]:
# Using lr.score simplifies the process of calculating accuracy by directly providing a single method to evaluate 
# the trained logistic regression model on a specific test dataset. It's a convenient way to assess the performance 
# of the model in terms of classification accuracy without needing to manually compare predicted and true labels.

lr.score(vect_X_test, y_test)

0.9866369710467706

In [15]:
accuracy_score(y_test, lr_y_pred)

0.9866369710467706

In [16]:
confusion_matrix(y_test, lr_y_pred)

array([[4671,   66],
       [  54, 4189]], dtype=int64)

In [17]:
from sklearn.metrics import classification_report
print(classification_report(y_test, lr_y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4737
           1       0.98      0.99      0.99      4243

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980



### Testing the model with a single input

In [18]:
def test(text):
    text = wordopt(text)
    text = vectorization.transform(pd.Series(text))
    prediction = lr.predict(text)

    if prediction == 0:
        return 'Fake News'
    elif prediction == 1:
        return 'Real News'

In [19]:
news = str(input())
test(news)

'Real News'

# Save the model

In [ ]:
# Import the pickle module
import pickle

# Save the linear regressor model to a file named RandomForestRegressor.plk
# Use the dump function to write the model object (rf) to the file in binary mode (wb)
pickle.dump(lr, open('../models/LinearRegressor.pkl', 'wb'))

# Save the linear regressor model to a file named RandomForestRegressor.plk
# Use the dump function to write the model object (rf) to the file in binary mode (wb)
pickle.dump(vectorization, open('../models/vectorization.pkl', 'wb'))

# # Load the random forest regressor model from the file named RandomForestRegressor.plk
# # Use the load function to read the model object from the file in binary mode (rb)
pickle_model = pickle.load(open('../models/vectorization.pkl', 'rb'))

# # Use the predict method of the model to make a prediction on the first row of the normalized feature matrix (X_norm)
# # Reshape the row into a 2D array with one row and -1 columns
# pickle_model.predict(X_norm[0].reshape(1,-1))

In [ ]:
text = "Indonesia to buy $1.14 billion worth of Russian jetsJAKARTA (Reuters) - Indonesia will buy 11 Sukhoi fighter jets worth $1.14 billion from Russia in exchange for cash and Indonesian commodities, two cabinet ministers said on Tuesday. The Southeast Asian country has pledged to ship up to $570 million worth of commodities in addition to cash to pay for the Suhkoi SU-35 fighter jets, which are expected to be delivered in stages starting in two years. Indonesian Trade Minister Enggartiasto Lukita said in a joint statement with Defence Minister Ryamizard Ryacudu that details of the type and volume of commodities were  still being negotiated . Previously he had said the exports could include palm oil, tea, and coffee. The deal is expected to be finalised soon between Indonesian state trading company PT Perusahaan Perdangangan Indonesia and Russian state conglomerate Rostec. Russia is currently facing a new round of U.S.-imposed trade sanctions. Meanwhile, Southeast Asia s largest economy is trying to promote its palm oil products amid threats of a cut in consumption by European Union countries. Indonesia is also trying to modernize its ageing air force after a string of military aviation accidents. Indonesia, which had a $411 million trade surplus with Russia in 2016, wants to expand bilateral cooperation in tourism, education, energy, technology and aviation among others."

In [ ]:
text = wordopt(text)
text

In [ ]:
# Import the pickle module
import pickle

In [ ]:
# Save the linear regressor model to a file named RandomForestRegressor.plk
# Use the dump function to write the model object (rf) to the file in binary mode (wb)
pickle.dump(vectorization, open('../models/vectorization.pkl', 'wb'))

In [ ]:
# # Load the random forest regressor model from the file named RandomForestRegressor.plk
# # Use the load function to read the model object from the file in binary mode (rb)
vec_model = pickle.load(open('../models/vectorization.pkl', 'rb'))

In [ ]:
textvec = vec_model.transform(pd.Series(text))


In [ ]:
textvec

In [ ]:
vectorization.transform(pd.Series(text))

In [ ]:
pred_model = pickle.load(open('../models/LinearRegressor.pkl', 'rb'))

In [ ]:
pred_model.predict([textvec].array.reshape(1,-1))

In [ ]:
def test2(text):
    text = wordopt(text)
    text = vec_model.transform(pd.Series(text))
    prediction = pred_model.predict(text)

    if prediction == 0:
        return 'Fake News'
    elif prediction == 1:
        return 'True as fuck'

In [ ]:
news = str(input())
test2(news)